In [15]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np

In [44]:
N = 1000
D = 100
batch_size = 128
ratio = 0.2
sigma_inv = 1
sigma_spu = 0.1
sigma_factor = [2, 20]



Z_1 = torch.normal(0, 1, (N, int(D*ratio))).repeat(len(sigma_factor)+1,1)
theta_1 = torch.normal(0, sigma_inv, (int(D*ratio), 1))
Y = ((Z_1 @ theta_1).squeeze() > 0).to(torch.float32) # Y is a binary vector with 0 and 1
Z_2 = [torch.normal(1/(D*(1-ratio))*(2*Y[:N]-1).unsqueeze(1).repeat(1,int(D*(1-ratio))), sigma_spu).to(torch.float32)]
for d, factor in enumerate(sigma_factor):
    Z_2.append((Z_2[0]-1/(D*(1-ratio))*(2*Y[:N]-1).unsqueeze(1))*factor+2/D*(2*Y[:N]-1).unsqueeze(1))
Z_2 = torch.cat(Z_2)
print(Z_1.shape, Z_2.shape, Y.shape)
Z = torch.cat((Z_1,Z_2),1)
while True:
    matrix = torch.rand(D, D)  # Create a DxD matrix with random values between 0 and 1
    if torch.linalg.matrix_rank(matrix) == D:
        Q, R = torch.linalg.qr(matrix)
        break
X = Z @ Q

X_train, X_test = X[:N*(len(sigma_factor))], X[N*(len(sigma_factor)):]
Y_train, Y_test = Y[:N*(len(sigma_factor))], Y[N*(len(sigma_factor)):]
Z_train, Z_test = Z[:N*(len(sigma_factor))], Z[N*(len(sigma_factor)):]





# ... (Your existing code for generating X, Y, Z) ...



class PairedDomainDataset(Dataset):
    def __init__(self, X, Y, Z, domain_labels):
        self.X = X
        self.Y = Y
        self.Z = Z
        self.domain_labels = domain_labels

    def __len__(self):
        return len(self.X) // 2  # Half the size since we're pairing

    def __getitem__(self, idx):
        # Get a pair of indices from the same domain
        idx1 = idx
        idx2 = idx + len(self.X) // 2
        return self.X[idx1], self.Y[idx1], self.X[idx2], self.Y[idx2]

# Generate domain labels (assuming the first half is one domain, the second half is the other)
domain_labels = np.concatenate([np.zeros(N * len(sigma_factor)), np.ones(N)])

# Create the paired dataset
paired_dataset = PairedDomainDataset(X_train, Y_train, Z_train, domain_labels)


paired_loader = DataLoader(paired_dataset, batch_size=batch_size, shuffle=True) 

loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True)


torch.Size([3000, 20]) torch.Size([3000, 80]) torch.Size([3000])


In [34]:
print(X_train.shape, Y_train.shape)

torch.Size([2000, 100]) torch.Size([2000])


In [35]:
# LogisticRegression
clf = LogisticRegression(random_state=0, fit_intercept=False).fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.783

In [36]:
# LogisticRegression
clf = LogisticRegression(random_state=0, fit_intercept=False).fit(Z_train, Y_train)
clf.score(Z_test, Y_test)

0.783

In [37]:
# Oracle
clf = LogisticRegression(random_state=0, fit_intercept=False).fit(Z_train[:,:int(D*ratio)], Y_train)
clf.score(Z_test[:,:int(D*ratio)], Y_test)

0.993

In [8]:
import torch.nn as nn
import torch.optim as optim

class LR(nn.Module):
    def __init__(self, D_in):
        super(LR, self).__init__()
        self.linear_1 = nn.Linear(D_in, D_in)
        self.linear_2 = nn.Linear(D_in, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.classifier(self.featurizer(x))

    def featurizer(self, x):
        x = self.linear_1(x)
        return x
    
    def classifier(self, x):
        x = self.linear_2(x)
        x = self.sigmoid(x)
        return x.squeeze()


In [9]:
for repeat in range(5):
    model = LR(D)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    for n in range(5000): 
        y = model(Z_train)
        loss = criterion(y, Y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    y_test = model(X_test)
    y_train = model(X_train)
    print(float(((y_train>0.5)==Y_train).sum()/len(y_train)), float(((y_test>0.5)==Y_test).sum()/len(y_test)))

0.4814999997615814 0.5049999952316284
0.48100000619888306 0.5040000081062317


KeyboardInterrupt: 

In [229]:
epoch = 2000
for repeat in range(5):
    model = LR(D)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    lmda = 1000
    model.train()
    for i in range(epoch):
        z_1 = model.featurizer(X_train[:N])
        z_2 = model.featurizer(X_train[N:])
        y_1 = model.classifier(z_1)
        y_2 = model.classifier(z_2)
        loss_1 = criterion(torch.cat((y_1,y_2)), Y_train)
        # print(z_1.shape,z_2.shape)
        loss_2 = (torch.norm(z_1-z_2, p=2) / len(z_1)) ** 2
        # print(loss_1,loss_2)
        loss = loss_1 + lmda * loss_2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    y_test = model(X_test)
    y_train = model(X_train)
    print(float(((y_train>0.5)==Y_train).sum()/len(y_train)), float(((y_test>0.5)==Y_test).sum()/len(y_test)))

KeyboardInterrupt: 

In [14]:
epoch = 10
for repeat in range(5):
    model = LR(D)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    lmda = 100
    model.train()
    for i in range(epoch):
        for j, (X_1, Y_1, X_2, Y_2) in enumerate(paired_loader):

            loss_1 = criterion(torch.cat((y_1,y_2)), torch.cat((Y_1,Y_2)))
            # print(z_1.shape,z_2.shape)
            loss_2 = (torch.norm(z_1-z_2, p=2) / len(z_1))
            # print(loss_1,loss_2)
            loss = loss_1 + lmda * loss_2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    model.eval()
    y_test = model(X_test)
    y_train = model(X_train)
    print(float(((y_train>0.5)==Y_train).sum()/len(y_train)), float(((y_test>0.5)==Y_test).sum()/len(y_test)))

0.9929999709129333 0.9900000095367432
0.9909999966621399 0.9900000095367432
0.9904999732971191 0.9919999837875366
0.996999979019165 0.9959999918937683
0.9919999837875366 0.9950000047683716


In [47]:
epoch = 20
shot = 60
for repeat in range(5):
    model = LR(D)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    lmda = 100
    model.train()
    for i in range(epoch):
        for j, (X, Y) in enumerate(loader):
            X= torch.cat((X_train[0:shot], X, X_train[N:N+shot]))
            Y = torch.cat((Y_train[0:shot], Y, Y_train[N:N+shot]))
            z = model.featurizer(X)
            y = model.classifier(z)
            loss_1 = criterion(y, Y)
            loss_2 = (torch.norm(z[:shot]-z[-shot:], p=2) / len(z_1))
            loss = loss_1 + lmda * loss_2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    model.eval()
    y_test = model(X_test)
    y_train = model(X_train)
    print(float(((y_train>0.5)==Y_train).sum()/len(y_train)), float(((y_test>0.5)==Y_test).sum()/len(y_test)))

0.9789999723434448 0.8700000047683716
0.9764999747276306 0.8690000176429749
0.9764999747276306 0.8809999823570251
0.9674999713897705 0.8489999771118164
0.9695000052452087 0.8519999980926514
